# IMPORTAR LIBRERIAS

In [ ]:
import glob 
import pandas as pd 
from datetime import datetime
import requests
from zipfile import ZipFile

# CARGA DEL ARCHIVO


In [ ]:
print('Iniciar descarga de archivos - Granda Xavier')
remote_url = 'http://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip'
local_file = 'datasource.zip'
data = requests.get(remote_url)
with open(local_file, 'wb') as f:
    f.write(data.content)
with ZipFile(local_file, 'r') as zipObj:
# Extrae todo el contenido del archivo zip en un directorio diferente.
    zipObj.extractall('dealership_data')

# EXTRAER

In [ ]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe

def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe    

def extract():
    extracted_data = pd.DataFrame(columns=['modelo_auto','anio_fabricacion','precio', 'combustible']) 
    #for csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
    #for json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    return extracted_data 

# TRANSFORMAR


In [ ]:
def transform(data):
    data['price'] = round(data.price, 2)
    return data


# CARGAR


In [ ]:
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

# Log

In [ ]:
def log(logfile, message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(logfile,"a") as f: 
        f.write('[' + timestamp + ']: ' + message + '\n')
        print(message)

# EJECUCION

In [ ]:
if __name__ == '__main__':

    logfile    = "dealership_logfile.txt"            
    # todos los eventos se registraran
    targetfile = "dealership_transformed_data.csv"   
    # Los datos transformados se registraran

    log(logfile, "ETL Trabajo iniciado")

    log(logfile, "Inicia Fase De Extraccion")
    extracted_data = extract()
    print(extracted_data)
    log(logfile, "Finaliza Fase De Extraccion")

    log(logfile, "Inicia Fase De Transformacion")
    transformed_data = transform(extracted_data)
    print(transformed_data)
    log(logfile, "Finaliza Fase De Transformacion")

    log(logfile, "Inicia Fase De Carga")
    load(targetfile, transformed_data)
    log(logfile, "Finaliza Fase De Carga")